In [ ]:
# default_exp models.FCNPlus

# FCNPlus

> This is an unofficial PyTorch implementation by Ignacio Oguiza - oguiza@gmail.com modified from:

* Wang, Z., Yan, W., & Oates, T. (2017, May). Time series classification from scratch with deep neural networks: A strong baseline. In 2017 international joint conference on neural networks (IJCNN) (pp. 1578-1585). IEEE.

* Fawaz, H. I., Forestier, G., Weber, J., Idoumghar, L., & Muller, P. A. (2019). Deep learning for time series classification: a review. Data Mining and Knowledge Discovery, 33(4), 917-963.
* FCN TensorFlow implementation: https://github.com/hfawaz/dl-4-tsc/blob/master/classifiers/fcn.py

* 👀 kernel filter size 8 has been replaced by 7 (I believe it's a bug)

In [ ]:
#export
from tsai.imports import *
from tsai.models.layers import *

In [ ]:
#export
class FCNPlus(Module):
    def __init__(self, c_in, c_out, layers=[128, 256, 128], kss=[7, 5, 3], coord=False, separable=False, 
                 zero_norm=False, act=nn.ReLU, act_kwargs={}, residual=False):
        assert len(layers) == len(kss)
        self.residual = residual
        self.convblock1 = ConvBlock(c_in, layers[0], kss[0], coord=coord, separable=separable, act=act, act_kwargs=act_kwargs)
        self.convblock2 = ConvBlock(layers[0], layers[1], kss[1], coord=coord, separable=separable, act=act, act_kwargs=act_kwargs)
        self.convblock3 = ConvBlock(layers[1], layers[2], kss[2], coord=coord, separable=separable, zero_norm=zero_norm if residual else False, 
                                    act=None if residual else act, act_kwargs=act_kwargs)
        if residual: self.shortcut = BN1(layers[2]) if c_in == layers[2] else ConvBlock(c_in, layers[2], 1, coord=coord, act=None) 
        self.add = Add() if residual else noop
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.squeeze = Squeeze(-1)
        self.fc = nn.Linear(layers[-1], c_out)

    def forward(self, x):
        if self.residual: res = x
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        if self.residual: x = self.add(x, self.shortcut(res))
        x = self.squeeze(self.gap(x))
        return self.fc(x)

In [ ]:
xb = torch.rand(16, 3, 10)
test_eq(FCNPlus(3, 2)(xb).shape, [xb.shape[0], 2])
test_eq(FCNPlus(3, 2, coord=True, separable=True, act=Swish, residual=True)(xb).shape, [xb.shape[0], 2])

In [ ]:
from tsai.models.FCN import *
test_eq(total_params(FCN(3,2)), total_params(FCNPlus(3,2)))

In [ ]:
FCNPlus(3,2)

FCNPlus(
  (convblock1): ConvBlock(
    (0): Conv1d(3, 128, kernel_size=(7,), stride=(1,), padding=(3,), bias=False)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (convblock2): ConvBlock(
    (0): Conv1d(128, 256, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (convblock3): ConvBlock(
    (0): Conv1d(256, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (gap): AdaptiveAvgPool1d(output_size=1)
  (squeeze): Squeeze()
  (fc): Linear(in_features=128, out_features=2, bias=True)
)

In [ ]:
#hide
out = create_scripts()
beep(out)